In [5]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from keras_image_helper import create_preprocessor

In [7]:
host = "localhost:8500"

# initializing channel, later we will use kubernetes and connection to tf-serving will be in internal part so we can use insecure channel
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [13]:
url = "http://bit.ly/mlbookcamp-pants"

preprocessor = create_preprocessor("xception", target_size=(299,299))

X = preprocessor.from_url(url)

In [16]:
pb_request = predict_pb2.PredictRequest()

#defining parameters using model-description: name, signature_name, name of the input
pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"

#function of converting numpy to protobuff
def np_2_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request.inputs["input_8"].CopyFrom(np_2_protobuf(X))

In [20]:
pb_response = stub.Predict(pb_request, timeout=20)

In [30]:
predicted_values = pb_response.outputs["dense_7"].float_val
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

result = dict(zip(labels, predicted_values))

{'dress': -1.8682893514633179,
 'hat': -4.761244773864746,
 'longsleeve': -2.3169822692871094,
 'outwear': -1.0625706911087036,
 'pants': 9.88715934753418,
 'shirt': -2.8124325275421143,
 'shoes': -3.6662826538085938,
 'shorts': 3.2003581523895264,
 'skirt': -2.6023366451263428,
 't-shirt': -4.835046291351318}